In [1]:
from xml.dom.minidom import Element
from PIL import Image
import pytesseract
from wand.image import Image as Img
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import numpy as np
import os
import cv2
import re
from distutils import core
from collections import defaultdict
import gspread
import time
import pandas as pd

In [2]:
extracted_text = ['SAT, IU SEY <Uee', '', 'RIVERSIDE GRILLED FISH SINGAPORE SG >', 'Dining', 'sep -74.15', 'WHITE RESTAURANT SINGAPORE SG >', 'Dining', 'scp -34.60', 'HYPER VIVO SINGAPORE SG >', 'Shopping', 'scp -9.65', '', 'FRI, 09 SEP 2022', '', 'CARD SHACK SINGAPORE SG >', 'Shopping', 'scp -8.88', 'LITTLE ROGUE COFFEE SINGAPORE SG >', 'Dining', 'sep -40.70', 'REINVENT VENTURES SINGAPORE SG >', 'Shopping', 'sco -5.00', '', 'REINVENT VENTURES SINGAPORE SG >', 'Jul) © Aug { sep ] Oct )( Nov )( Current ) | t', '', 'SGD -1.3U0', 'NET*LINCO SINGAPORE SG >', 'Dining', '', 'sop -1.70', 'NET*LINCO SINGAPORE SG >', 'Dining', '', 'sop -1.20', 'NET*LINCO SINGAPORE SG 2', 'Dining', '', 'scp -1.20', 'TUE, 06 SEP 2022', 'BUS/MRT 179030552 SINGAPORE SG >', 'Transportation', '', 'sep -2.82', 'NET*LINCO SINGAPORE SG >', 'Dining', '', 'scp -1.30', 'NET*LINCO SINGAPORE SG >', 'Dining', 'Jul) © Aug { sep ] Oct )( Nov) Current ) | t', '', 'NET*LINCO SINGAPORE SG >', '', 'Dining', 'sop -1.20', 'SUN, 04 SEP 2022', '', 'BUS/MRT 178475165 SINGAPORE SG >', '', 'Transportation', '', 'sop -1.90', '', 'SAT, 03 SEP 2022', '', 'BUS/MRT 178160838 SINGAPORE SG >', 'Transportation', '', 'sop -2.23', 'WHITE RESTAURANT SINGAPORE SG >', 'Dining', '', 'scp -39.90', '', "WATSON'S SINGAPORE SG >", 'Health & Fitness', '', 'scp -2.50', 'PET LOVERS CENTRE PTE SINGAPORE SG 7', '', 'Shopping', 'Jul) ( Aug { sep ] Oct )( Nov )( Current ) | t', '', 'vining', '', 'scp -40.70', 'REINVENT VENTURES SINGAPORE SG >', 'Shopping', 'scp -5.00', 'REINVENT VENTURES SINGAPORE SG >', 'Shopping', 'scp -6.93', 'THU, 08 SEP 2022', 'PET LOVERS CENTRE PTE SINGAPORE SG >', 'Shopping', 'sep -47.25', 'NET*LINCO SINGAPORE SG >', 'Dining', 'sop -1.30', 'WED, 07 SEP 2022', 'BUS/MRT 179244836 SINGAPORE SG >', '', 'Transportation', '', 'scp -2.82']

In [3]:
#Regex 
date_pattern = re.compile(r'^(?:MON|TUE|WED|THU|FRI|SAT|SUN), (.|..) (?:JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC) (?:2022|2023)$')
amount_pattern = re.compile(r'\d*,*\d*\.\d*')

Remove any items before the first date as transactions only come after dates

In [4]:
firstDateIndex = None
for index, element in enumerate(extracted_text):
    if date_pattern.match(element):
        firstDateIndex = index
        break

In [5]:
clean_extracted_text = extracted_text[firstDateIndex: len(extracted_text)]
print(clean_extracted_text)

['FRI, 09 SEP 2022', '', 'CARD SHACK SINGAPORE SG >', 'Shopping', 'scp -8.88', 'LITTLE ROGUE COFFEE SINGAPORE SG >', 'Dining', 'sep -40.70', 'REINVENT VENTURES SINGAPORE SG >', 'Shopping', 'sco -5.00', '', 'REINVENT VENTURES SINGAPORE SG >', 'Jul) © Aug { sep ] Oct )( Nov )( Current ) | t', '', 'SGD -1.3U0', 'NET*LINCO SINGAPORE SG >', 'Dining', '', 'sop -1.70', 'NET*LINCO SINGAPORE SG >', 'Dining', '', 'sop -1.20', 'NET*LINCO SINGAPORE SG 2', 'Dining', '', 'scp -1.20', 'TUE, 06 SEP 2022', 'BUS/MRT 179030552 SINGAPORE SG >', 'Transportation', '', 'sep -2.82', 'NET*LINCO SINGAPORE SG >', 'Dining', '', 'scp -1.30', 'NET*LINCO SINGAPORE SG >', 'Dining', 'Jul) © Aug { sep ] Oct )( Nov) Current ) | t', '', 'NET*LINCO SINGAPORE SG >', '', 'Dining', 'sop -1.20', 'SUN, 04 SEP 2022', '', 'BUS/MRT 178475165 SINGAPORE SG >', '', 'Transportation', '', 'sop -1.90', '', 'SAT, 03 SEP 2022', '', 'BUS/MRT 178160838 SINGAPORE SG >', 'Transportation', '', 'sop -2.23', 'WHITE RESTAURANT SINGAPORE SG >', '

In [6]:
indexOfDates = []
dates = []
indexOfPrices = []
prices = []
indexOfTransactions = []
trans = []

for index, element in enumerate(clean_extracted_text):
    if date_pattern.match(element):
        indexOfDates.append(index)
        dates.append(element)
    if amount_pattern.search(element):
        indexOfPrices.append(index)
        prices.append(element)
    if bool(date_pattern.match(element)) == False and bool(amount_pattern.search(element)) == False and element != '':
        indexOfTransactions.append(index)
        trans.append(element)

print(list(prices))

['scp -8.88', 'sep -40.70', 'sco -5.00', 'SGD -1.3U0', 'sop -1.70', 'sop -1.20', 'scp -1.20', 'sep -2.82', 'scp -1.30', 'sop -1.20', 'sop -1.90', 'sop -2.23', 'scp -39.90', 'scp -2.50', 'scp -40.70', 'scp -5.00', 'scp -6.93', 'sep -47.25', 'sop -1.30', 'scp -2.82']


Removing text from front of amount

In [7]:
credit_or_debit = []
amountList = []

positive_pattern = re.compile(r'\+')
negative_pattern = re.compile(r'\-')
dollars_pattern = re.compile(r'(\d+?,?\d+\.\d+)')
text_pattern = re.compile(r'(scp|sep)\s(\-|\+)')

for i in prices:
    if positive_pattern.search(i):
        credit_or_debit.append('credit')
        i = re.sub(text_pattern, '', i)
        amountList.append(i)
    if negative_pattern.search(i):
        credit_or_debit.append('debit')
        i = re.sub(text_pattern, '', i)
        amountList.append("("+i+")")

print(len(credit_or_debit))
print(len(amountList))
print(amountList)

20
20
['(8.88)', '(40.70)', '(sco -5.00)', '(SGD -1.3U0)', '(sop -1.70)', '(sop -1.20)', '(1.20)', '(2.82)', '(1.30)', '(sop -1.20)', '(sop -1.90)', '(sop -2.23)', '(39.90)', '(2.50)', '(40.70)', '(5.00)', '(6.93)', '(47.25)', '(sop -1.30)', '(2.82)']


Make multiple strings of one transaction into a single string

In [8]:
#Joining separate transactions
newTransIndex =[]
joinedTrans = []
placeholderTransIndex = 0

for i in indexOfPrices:
    combinedLine = ""
    for index, element in enumerate(clean_extracted_text):
        if bool(date_pattern.match(element)) == False and bool(amount_pattern.search(element)) == False and element != '' and index < i and index > placeholderTransIndex:
            combinedLine += element.lower()
    joinedTrans.append(combinedLine)
    newTransIndex.append(i - 1)
    placeholderTransIndex = i - 1

print(joinedTrans)


['card shack singapore sg >shopping', 'little rogue coffee singapore sg >dining', 'reinvent ventures singapore sg >shopping', 'reinvent ventures singapore sg >jul) © aug { sep ] oct )( nov )( current ) | t', 'net*linco singapore sg >dining', 'net*linco singapore sg >dining', 'net*linco singapore sg 2dining', 'bus/mrt 179030552 singapore sg >transportation', 'net*linco singapore sg >dining', 'net*linco singapore sg >diningjul) © aug { sep ] oct )( nov) current ) | tnet*linco singapore sg >dining', 'bus/mrt 178475165 singapore sg >transportation', 'bus/mrt 178160838 singapore sg >transportation', 'white restaurant singapore sg >dining', "watson's singapore sg >health & fitness", 'pet lovers centre pte singapore sg 7shoppingjul) ( aug { sep ] oct )( nov )( current ) | tvining', 'reinvent ventures singapore sg >shopping', 'reinvent ventures singapore sg >shopping', 'pet lovers centre pte singapore sg >shopping', 'net*linco singapore sg >dining', 'bus/mrt 179244836 singapore sg >transportat

In [9]:
#Create dict for dates and its indexes  
datesCopy = dates.copy()
dateDict = {}
for key in indexOfDates:
    for value in datesCopy:
        dateDict[key] = value
        datesCopy.remove(value)
        break

In [10]:
#Matching dates to transactions
listDates = []
minDate = indexOfDates[0]
nextDate = indexOfDates[0]

for i in range(0, len(indexOfPrices)):
    if indexOfPrices[i] > indexOfDates[-1]:
        listDates.append(dateDict[indexOfDates[-1]])
    for j in range(0, len(indexOfDates) -1):
        if indexOfPrices[i] > indexOfDates[j + 1]:
            minDate = indexOfDates[j + 1]
        if indexOfPrices[i] < indexOfDates[j + 1] and indexOfPrices[i] > minDate:
            listDates.append(dateDict[minDate])
            nextDate = indexOfDates[j + 1]
            break;


if len(newTransIndex) == len(indexOfPrices) == len(listDates) == len(credit_or_debit) == len(amountList):
    pass
else: 
    print("number of records are not the same")


#Check the items in each list matches 
print(len(newTransIndex))
print(len(indexOfPrices))
print(len(listDates))

print("Original Extracted Text --- ", extracted_text)
print("")
print(joinedTrans)
print("")
print(amountList)
print("")
print(listDates)


20
20
20
Original Extracted Text ---  ['SAT, IU SEY <Uee', '', 'RIVERSIDE GRILLED FISH SINGAPORE SG >', 'Dining', 'sep -74.15', 'WHITE RESTAURANT SINGAPORE SG >', 'Dining', 'scp -34.60', 'HYPER VIVO SINGAPORE SG >', 'Shopping', 'scp -9.65', '', 'FRI, 09 SEP 2022', '', 'CARD SHACK SINGAPORE SG >', 'Shopping', 'scp -8.88', 'LITTLE ROGUE COFFEE SINGAPORE SG >', 'Dining', 'sep -40.70', 'REINVENT VENTURES SINGAPORE SG >', 'Shopping', 'sco -5.00', '', 'REINVENT VENTURES SINGAPORE SG >', 'Jul) © Aug { sep ] Oct )( Nov )( Current ) | t', '', 'SGD -1.3U0', 'NET*LINCO SINGAPORE SG >', 'Dining', '', 'sop -1.70', 'NET*LINCO SINGAPORE SG >', 'Dining', '', 'sop -1.20', 'NET*LINCO SINGAPORE SG 2', 'Dining', '', 'scp -1.20', 'TUE, 06 SEP 2022', 'BUS/MRT 179030552 SINGAPORE SG >', 'Transportation', '', 'sep -2.82', 'NET*LINCO SINGAPORE SG >', 'Dining', '', 'scp -1.30', 'NET*LINCO SINGAPORE SG >', 'Dining', 'Jul) © Aug { sep ] Oct )( Nov) Current ) | t', '', 'NET*LINCO SINGAPORE SG >', '', 'Dining', 'so